In [1]:
import os
import sys
from glob import glob

In [2]:
import shutil
import zipfile

In [3]:
class MoveFile:
    """想定するファイル構成

    <DataSet RootDir>
        |
        |-- <original>

    """
    def __init__(self, parent_path):
        # 操作したいデータセットのrootディレクトリ
        self.parent_path = parent_path
        # 
        self.org_path = os.path.join(self.parent_path, 'original')

    def unpack(self, file_name: str, create_dir: bool=True):
        """originalディレクトリ内のzipファイルを展開するための関数
        Arg:
            file_name(str): original ディレクトリ内の zip ファイル名,
            create_dir(bool optional): 解凍するときに、解凍前のファイル名と同じ名前のディレクトリを作成する。default to True.
        """
        zip_path = os.path.join(self.org_path, file_name + '.zip')
        unpack_path = self.parent_path

        if create_dir:
            try:
                unpack_path = os.path.join(self.parent_path, file_name)
                os.mkdir(unpack_path) # create directry
            except FileExistsError as e:
                print("ディレクトリ作成時にエラーが発生しました: ", e)
                return
            
        with zipfile.ZipFile(zip_path) as existing_zip:
            existing_zip.extractall(unpack_path)

    def MoveImage(self, src: str, dst: str):
        """子ディレクトリ内に存在するファイルを別の子ディレクトリ内の正数で表されたディレクトリ内に移動する関数
        Args:
            src(str): 移動させたいファイルやディレクトリのパス,
            dst(str): 移動先のディレクトリのパス

        <Root Dir>
        |-- 0a7d3O
        |   |-- images
        |   |     |-- 0a7d3O.png
        |   |
        |   |-- masks
        |         |-- 0adbf5.png
        |         |-- 1ed5b9.png
        |         |-- 3bef1O.png
        |         |-- ...
        |
        |-- 0acd2c
        |   |-- images
        |   |     |-- 0acd2c.png
        |   |
        |   |-- masks
        |         |-- 2f5eb1.png
        |         |-- 3fa2a8.png
        |         |-- O5fefb.png
        |         |-- ...
        |
        |-- 00ae65c
        |   |-- ...
        |
        |-- ...
        """
        # 出力先のディレクトリが存在するか判定
        if not os.path.exists(dst): # 存在しない場合
            os.mkdir(dst)

        # 出力先の images ディレクトリを作成する
        dst_img_path = makedir(dst, 'images')

        # 出力先の masks ディレクトリを作成する
        dst_mask_path = makedir(dst, 'masks')

        # 全ての子ディレクトリの相対パスを取得 <Root> -> <0a7d330>
        list_dir_path = glob(os.path.join(src, '**'))
        for dir_path in list_dir_path:
            # 親ディレクトリ名を取得
            dir_name = os.path.basename(dir_path.rstrip(os.sep))

            # 子ディレクトリ内の images ディレクトリの相対パスを取得 <0a7d330> -> images
            list_img_dir_path = [f for f in glob(os.path.join(dir_path, '**')) if 'images' in f]
            for img_dir_path in list_img_dir_path:
                list_file = os.listdir(img_dir_path)
                if list_file: # ディレクトリ内にファイルが存在するか判定
                    for p in list_file:
                        # ディレクトリ内の全てのファイルを移動
                        shutil.move(os.path.join(img_dir_path, p), dst_img_path)

            # 子ディレクトリ内の masks ディレクトリの相対パスを取得 <0a7d330> -> masks
            list_masks_dir_path = [f for f in glob(os.path.join(dir_path, '**')) if 'masks' in f]
            for masks_dir_path in list_masks_dir_path:
                list_file = os.listdir(masks_dir_path)
                if list_file: # ディレクトリ内にファイルが存在するか判定
                    for num, p in enumerate(list_file):
                        old_name = os.path.join(masks_dir_path, p)
                        # 拡張子のみ取得
                        _, ext = os.path.splitext(p)
                        new_name = os.path.join(masks_dir_path, dir_name+ext) # C\\dataset\\masks\\label_name + \\label_name + .png

                        if not (p in new_name): # 同名のファイルがない場合
                            os.rename(old_name, new_name) # ファイルリネーム
                        else:  new_name = old_name

                        dst_path = os.path.join(dst_mask_path, str(num))
                        if not os.path.exists(dst_path):
                            os.mkdir(dst_path)
                        # ディレクトリ内のファイルを移動
                        shutil.move(new_name, dst_path)

In [4]:
sys.path.append(".")
sys.path.append("..")
sys.path.append("../../")
from NestedUNet.utils import makedir
from src.config.config import cfg

In [5]:
m_file = MoveFile(cfg.DATA_SCIENCE_BOWL_DIR)

In [6]:
# f_name = 'stage1_sample_submission.csv' # 解凍したいファイル名
f_name = 'stage1_train'
m_file.unpack(f_name)

ディレクトリ作成時にエラーが発生しました:  [WinError 183] 既に存在するファイルを作成することはできません。: 'd:\\DATA\\R2_Miki\\python\\AI\\Analysis\\data\\DataScienceBowl\\stage1_train'


In [7]:
src = os.path.join(cfg.DATA_SCIENCE_BOWL_DIR, 'stage1_train')
dst = cfg.DSB2018_96_DIR
m_file.MoveImage(src, dst)

# ディレクトリ名を取得するテスト

In [8]:

src = os.path.join(cfg.DATA_SCIENCE_BOWL_DIR, 'stage1_train', '**')

for p in glob(src):
    dir_name = os.path.basename(p.rstrip(os.sep))
    print(dir_name)

3d60acaaf33af45ef4fbffaac15ae838bc1ce37b6fbf
8b12e18670e4b24d03567d1e17c0c24fadf0ea2c1e763983dd6bb4c44b7376a6
8b77284d6f37ab3fc826139ebadaec3b9d81c552fe525c3547bbbd6c65ac0d83
8bef203fce625e4d8c89dca728158be4662dfdfdcd4dc73a6aa39a908c1631bc
8c3ef7aa7ed29b62a65b1c394d2b4a24aa3da25aebfdf3d29dbfc8ad1b08e95a
8cdbdda8b3a64c97409c0160bcfb06eb8e876cedc3691aa63ca16dbafae6f948
8d05fb18ee0cda107d56735cafa6197a31884e0a5092dc6d41760fb92ae23ab4
8d29c5a03e0560c8f9338e8eb7bccf47930149c8173f9ba4b9279fb87d86cf6d
8d9b4205ddb10fa49a2973b4f3a2dc6923407ae015081e1a52c4b4c2fe8faa53
8de0b1a2e8f614af29fe5fafeaa5bdf55e6b3e65edf36355f19b707f7649ce2b
8e507d58f4c27cd2a82bee79fe27b069befd62a46fdaed20970a95a2ba819c7b
8e8a7a14749d0b2e48de3d10e2e80063f17b165ad921c8afc0623f08500f3259
8ecdb93582b2d5270457b36651b62776256ade3aaa2d7432ae65c14f07432d49
8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177ef25a9dee25ffe8842
8f27ebc74164eddfe989a98a754dcf5a9c85ef599a1321de24bcf097df1814ca
8f6597cd978c060378177df76e554d0578b97eab471e2

# ファイルリネームのテスト

In [9]:
dst = cfg.DSB2018_96_DIR
dst_mask_path = os.path.join(dst, 'masks')
try:
    os.mkdir(dst_mask_path)
except FileExistsError as e:
    print(e)
    pass

list_dir_path = glob(os.path.join(src, '**'))
dir_path = list_dir_path[0]
# 親ディレクトリ名を取得
dir_name = os.path.basename(dir_path.rstrip(os.sep))

# 子ディレクトリ内の masks ディレクトリの相対パスを取得 <0a7d330> -> masks
list_masks_dir_path = [f for f in glob(os.path.join(dir_path, '**')) if 'masks' in f]
masks_dir_path = list_masks_dir_path[0]
list_file = os.listdir(masks_dir_path)
p = list_file[0]

print('-'*20)
print('dir_name: ', dir_name)
print('masks_dir_path: ', masks_dir_path)
print('lsit_file: ', p)
print('-'*20)

[WinError 183] 既に存在するファイルを作成することはできません。: 'd:\\DATA\\R2_Miki\\python\\AI\\Analysis\\data\\DataScienceBowl\\dsb2018_96\\masks'


IndexError: list index out of range

In [130]:
old_name = os.path.join(masks_dir_path, p)
# 拡張子のみ取得
_, ext = os.path.splitext(p)
new_name = os.path.join(masks_dir_path, dir_name+ext)

if not (p in new_name): # 同名のファイルがない場合
    # ファイルリネーム
    p = os.rename(old_name, new_name)
    print('rename is: ', p)
else: # 同名のファイルがある場合
    print("既に存在するファイル名です。")

既に存在するファイル名です。
